# LCLUC project notebook - Classifying phase

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import rasterio
from osgeo import gdal

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from skimage.filters.rank import modal
from skimage.morphology import square

import pickle

In [2]:
path = '136'
year = '2005'
wdir = 'd:/CSISS/data/' + path + '/' + year
os.chdir(wdir)

In [3]:
smooth_times = 3

In [4]:
data = rasterio.open('mosaic.tif')

In [5]:
def Classify(file, clf):
    data = rasterio.open(file)
    mask = np.ones(data.shape, dtype=bool)
    for i in range(1, data.count+1):
        band = data.read(i)
        tmp_mask = (band > 0)
        mask = (mask & tmp_mask)
    
    #data_masked = []
    data_m = np.empty(shape=(data.count, data.shape[0], data.shape[1]), dtype='int')
    for i in range(1, data.count+1):
        band = data.read(i).copy()
        band[~mask] = 0
        #data_masked.append(band)
        data_m[i-1] = band
        
    #data_m = np.array(data_masked, dtype='int')
    data_m = data_m.swapaxes(0, 2)
    
        
    X_data = np.reshape(data_m, (data_m.shape[0]*data_m.shape[1], data_m.shape[2]), order='C')
    X_df = pd.DataFrame(X_data)
   
    X_target = X_df[(X_df.iloc[:, 0] != 0) & (X_df.iloc[:, 1] != 0) & (X_df.iloc[:, 2] != 0) &
                   (X_df.iloc[:,3]!=0) & (X_df.iloc[:,4]!= 0) & (X_df.iloc[:,5]!=0)].copy()
    del X_df
    
    b1 = X_target.iloc[:, 0]
    b2 = X_target.iloc[:, 1]
    b3 = X_target.iloc[:, 2]
    b4 = X_target.iloc[:, 3]
    b5 = X_target.iloc[:, 4]
    b6 = X_target.iloc[:, 5]

    
    X_target['ndvi'] = (b4 - b3)/(b4 + b3)
    X_target['ndwi'] = (b2 - b4)/(b2 + b4)
    X_target['ndbi'] = (b5 - b4)/(b4 + b5)
    
    X_target['savi'] = 1.5*(b4 - b3)/(b4 + b3 + 0.5)
    X_target['ndsi'] = (b6 - b2)/(b6 + b2)
    X_target['mndwi'] = (b2 - b5)/(b2 + b5)
    X_target['evi'] = 2.5*(b4 - b3)/(b4 + 6*b3 - 7.5*b1 + 1)
   
    X_target['ui'] = (b6 - b4)/(b6 + b4)
    X_target[(X_target.iloc[:, -2] == np.inf)] = 10000
    X_target[(X_target.iloc[:, -2] == -np.inf)] = -10000
    X_target[np.isnan(X_target.iloc[:, -2])] = 10000
    
    X_array = X_target.as_matrix()
    
    print(np.any(np.isnan(X_target.iloc[:, :6])))
    print(np.any(np.isnan(X_target.iloc[:, -2:])))
    
    
    y_array = np.zeros(shape=(X_array.shape[0], ))
    for i in range(int(len(X_array)/5000000) +1):
        if (i+1)*5000000 <= X_array.shape[0]:
            X_slice = X_array[i*5000000:(i+1)*5000000, :]
            y_predict = clf.predict(X_slice)
            y_array[i*5000000:(i+1)*5000000] = y_predict
        else:
            X_slice = X_array[i*5000000:, :]
            y_predict = clf.predict(X_slice)
            y_array[i*5000000:] = y_predict
            
    y_array = y_array.astype('uint8')
    result = np.empty(shape=(X_data.shape[0],), dtype='uint8')
    for i, ind in enumerate(X_target.index):
        result[ind] = y_array[i]
    result_2d = np.reshape(result, (data_m.shape[0], data_m.shape[1]), order='C')
    result_2d = result_2d.swapaxes(0,1)
    
    ##### Smooth Results ######
    def SmoothImage(img):
        smoothed = modal(img, selem=square(3), mask=mask)
        return smoothed
    
    for i in range(smooth_times):
        result_2d = SmoothImage(result_2d)
    
    ###########################
    result_file = rasterio.open(file[:-4] + '_output.tif', 'w', driver='GTiff', height=result_2d.shape[0], width=result_2d.shape[1], count=1, 
                         dtype='uint8', crs=data.crs, transform=data.transform)
    result_file.write(result_2d, 1)
    result_file.close()
    print('Done for ' + file)
    del data, data_m, mask, X_target

In [6]:
targetfiles = glob.glob('*stack.tif')
clf_file = open('clf.pkl', 'rb')
clf = pickle.load(clf_file)
for file in targetfiles:
    Classify(file, clf)


False
False


C:\Users\Zhiqi\Miniconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  """
C:\Users\Zhiqi\Miniconda3\envs\geo\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


Done for 1360432004020801_stack.tif
False
False
Done for 1360442004020801_stack.tif
False
False
Done for 1360452004020801_stack.tif
